In [4]:
## 吐き出されたcsvのy座標は上からになってることに注意
##size[1]から引く

In [5]:
import cv2
import csv
import math
import numpy as np
import datetime
import os
from datetime import datetime, timedelta,timezone
import json
import re
import gzip
import sys
from tqdm.notebook import tqdm_notebook as loop
import pandas as pd

In [6]:
#AR認識準備
dictionary_name = cv2.aruco.DICT_6X6_250
dictionary = cv2.aruco.getPredefinedDictionary(dictionary_name)

idMax = 4 #マーカの数

def getMarkerMean(ids, corners, index):
    for i, id in enumerate(ids):
        # マーカーのインデックス検索
        if(id[0] == index):
            v = np.mean(corners[i][0],axis=0) # マーカーの四隅の座標から中心の座標を取得する
            return [v[0],v[1]]
    return None

def getBasisMarker(ids, corners):
    # 左上、右上、左下、右下の順にマーカーの「中心座標」を取得
    basis = []
    for i in range(idMax):
        tmp=getMarkerMean(ids,corners,i)
        if tmp is not None:
            basis.append(tmp)
    return basis

def getTransformImage(basis,frame,size):
    basis_coordinates = np.float32(basis)
    target_coordinates   = np.float32([[0, 0],[size[0], 0],[0, size[1]],[size[0], size[1]]])
    trans_mat = cv2.getPerspectiveTransform(basis_coordinates,target_coordinates)
    return cv2.warpPerspective(frame, trans_mat, (size[0], size[1]))

def getTransformedPoint(basis, size,x,y):
    basis_coordinates = np.float32(basis)
    target_coordinates   = np.float32([[0, 0],[size[0], 0],[0, size[1]],[size[0], size[1]]])
    trans_mat = cv2.getPerspectiveTransform(basis_coordinates,target_coordinates)
    temp=np.array([[x],[y],[1]])
    transformed_temp=np.dot(trans_mat,temp)
    return transformed_temp[:2]/transformed_temp[2]

In [7]:
#注目座標データ読み込み
def readGazefile(filename):
    with open(filename) as f:
        reader = csv.reader(f)
        l = [row for row in reader]
    return l

In [20]:
# 動画保存準備
def readyVideoCapture(fps,size):
    fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    filename= os.path.join(root,"fixed_video.mp4")
    writer = cv2.VideoWriter(filename, fmt, fps, size)
    return writer

def gaze2D2framepoint(l,gazedata_idx,size):
    gaze_x=float(l[gazedata_idx][1])
    gaze_y=float(l[gazedata_idx][2])
    x=int(size[0]*gaze_x)
    y=int(size[1]*gaze_y)
    return (x,y)

In [21]:
def saveGazepoint(basis,frame,size,x,y,save_num,transformedpoints):
#     frame2 = getTransformImage(basis,frame,size)#射影変換
    frame2=frame
    if x>0:
        transformed_point=getTransformedPoint(basis,size,x,y)#変換後の座標
#         transformedpoints[save_num]=transformed_point
        transformedpoints[save_num]=np.transpose(transformed_point)
    else:
        transformedpoints[save_num]=(-1,-1)
    save_num+=1
    return frame2,save_num,transformedpoints

In [22]:
def autoadjust(frame):
    cols, rows, temp = frame.shape
    brightness = np.sum(frame) / (255 * cols * rows*3)
    minimum_brightness = 0.5
    ratio = brightness / minimum_brightness
    if ratio >= 1:
        return frame
    return cv2.convertScaleAbs(frame, alpha = 1 / ratio, beta = 0)

In [23]:
def main(gazefilename,videofilename,out_path):
    #動画読込
    cap=cv2.VideoCapture(videofilename)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    count = cap.get(cv2.CAP_PROP_FRAME_COUNT);
    size = (width, height)
    #視線データ読み込み
    l=readGazefile(gazefilename)

    if (cap.isOpened()== False):  print("ビデオファイルを開くとエラーが発生しました") 

    writer=readyVideoCapture(fps,size)#動画保存する場合

    transformedpoints=np.zeros((100000,2))
    save_num=0
    now=0
    while(cap.isOpened()):
        now=now+1/fps
        gazedata_idx=int(now/0.02)#数字はgaze2dの時間間隔．50Hz-->0.02

        ret, frame = cap.read()

        if(frame is None):
            if(now>300):
                break
            else:
                continue

        if len(l)<=gazedata_idx:
            break
            
        #映像の明るさとコントラスト自動修正
        frame=autoadjust(frame)
        
        #注目点座標
        if(not math.isnan(float(l[gazedata_idx][1]))):
            (x,y)=gaze2D2framepoint(l,gazedata_idx,size)
            cv2.rectangle(frame,(x-5,y-5),(x+5,y+5),color=(0,0,255),thickness=2)##赤い四角描画

        #ARマーカ認識
        corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(frame, dictionary)
        frame=cv2.aruco.drawDetectedMarkers(frame,corners,ids)##マーカー検知描画

        #射影変換，注目点座標保存
        if ids is not None:
            detectCount = len(ids)
            basis=getBasisMarker(ids,corners)
            if len(basis)==idMax:##ARマーカが４つあるとき
                frame,save_num,transformedpoints=saveGazepoint(basis,frame,size,x,y,save_num,transformedpoints)
            else:
                transformedpoints[save_num]=(-1,-1)
                save_num+=1
        else:##ARマーカがない
            transformedpoints[save_num]=(-1,-1)
            save_num+=1


    writer.write(frame)#動画保存
#         if ret == True:
#             frame = cv2.resize(frame, dsize=(800,450))
#             cv2.imshow("Video", frame)#動画表示
#             if cv2.waitKey(25) & 0xFF == ord('q'): 
#                 break
#         else:
#             break

    np.savetxt(out_path, transformedpoints,delimiter=',', fmt='%d')
    writer.release()#動画保存終わり
    cap.release()
    cv2.destroyAllWindows()

In [24]:
root="./data/test_ar/20220610T063504Z/"

In [25]:
main(os.path.join(root,"gaze.csv"),os.path.join(root,"scenevideo.mp4"),os.path.join(root,"fixed_video.csv"))

## Gaze 座標

In [11]:
def extract_timestamp(path):
    time_str = os.path.basename(os.path.dirname(path)).split("_")[0]
    dt = datetime.strptime(time_str,'%Y%m%dT%H%M%SZ')
    timestamp = dt.replace(tzinfo=timezone.utc).timestamp()
    return timestamp
    

def read_tobii_data(file_path):
    start_time = extract_timestamp(file_path)
    data = []
    with gzip.open(file_path,'r') as fin:        
        for line in fin:        
            tmp = str(line)[2:-3]
            obj = json.loads(tmp)
            data.append(obj)
    gazeX = []
    gazeY = []
    time_axis = []
    for d in loop(data):
        if not "data" in d or not "gaze2d" in d["data"]:continue
        time_axis.append(d['timestamp']+start_time)
        gazeX.append(d['data']['gaze2d'][0])
        gazeY.append(d['data']['gaze2d'][1])
    #df = pd.DataFrame(np.array([time_axis,p_left,p_right,gazeX,gazeY]).T,columns=["TIME","pLeft","pRight","gazeX","gazeY"])
    df = pd.DataFrame(np.array([time_axis,gazeX,gazeY]).T,columns=["TIME","gazeX","gazeY"])
    df.to_csv(os.path.join(os.path.dirname(file_path),"gaze.csv"),index=False,header=False)
    return df

In [13]:
df = read_tobii_data("./data/test_ar/20220610T063504Z/gazedata.gz")

  0%|          | 0/2973 [00:00<?, ?it/s]